# 🎯 Insurance ABAC Demo - Step 1: Create Masking Functions

## 📋 Overview
This notebook creates **masking functions** for the Insurance industry ABAC (Attribute-Based Access Control) demo.

### What are Masking Functions?
Masking functions are SQL user-defined functions (UDFs) that transform sensitive data to protect privacy while maintaining data utility for analytics. They are the foundation of ABAC policies in Unity Catalog.

### Why Use Masking Functions?
- **Compliance**: Meet GDPR, CCPA, HIPAA, and other privacy regulations
- **Security**: Protect sensitive data from unauthorized access
- **Flexibility**: Apply different masks based on user roles and attributes
- **Analytics**: Preserve data utility for analysis while protecting privacy
- **Audit**: Track and log all data access patterns

### What This Notebook Creates
This notebook will create specialized masking functions for the Insurance industry, including:
- **Identity Protection**: Email, phone, address masking
- **Financial Data**: Credit card, transaction amount bucketing
- **Identifiers**: Deterministic hashing for cross-table analytics
- **Confidential Data**: Complete redaction of sensitive fields
- **Network Data**: IP address masking

## 🎓 How to Use This Notebook
1. **Update Configuration**: Change the catalog name in the configuration cell below
2. **Run All Cells**: Execute cells sequentially (Shift+Enter or Run All)
3. **Verify Success**: Check for ✅ success messages after each function
4. **Proceed to Next Step**: Continue to notebook 2 to create the schema

## ⚙️ Prerequisites
- ✅ Unity Catalog enabled workspace
- ✅ CREATE FUNCTION permission in the target catalog
- ✅ SQL Warehouse or Cluster attached to this notebook
- ✅ Account admin or catalog owner role (recommended)

## 🔄 Next Steps
After completing this notebook:
1. **Step 2**: `2_Create_Schema.ipynb` - Create database schema and core tables
2. **Step 3**: `3_Create_Extended_Tables.ipynb` - Add supplementary tables
3. **Step 4**: `4_Test_Masking.ipynb` - Test all masking functions

---


## ⚙️ Configuration

### 🚨 IMPORTANT: Update Before Running!
Change `your_catalog_name` to **your catalog name** in the cell below or update the config.yaml.

### What This Does:
- Sets the target Unity Catalog
- Creates the `insurance` schema if it doesn't exist



In [0]:
%python
# 📋 Load Configuration from config.yaml
import yaml
from pathlib import Path

config_file = Path('config.yaml')
if config_file.exists():
    with open(config_file) as f:
        config = yaml.safe_load(f)
    CATALOG = config['catalog']
    SCHEMA = config['schema']
    print(f'✅ Configuration loaded from config.yaml')
    print(f'   📊 Catalog: {CATALOG}')
    print(f'   📁 Schema: {SCHEMA}')
else:
    # Fallback defaults
    CATALOG = 'your_catalog_name'
    SCHEMA = 'insurance'
    print(f'⚠️  config.yaml not found - using defaults')
    print(f'   📊 Catalog: {CATALOG}')
    print(f'   📁 Schema: {SCHEMA}')

# Set catalog and schema to use for the cells below
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA}")
spark.sql(f"USE SCHEMA {SCHEMA}")


In [0]:
SELECT '🎯 Target: ' || current_catalog() || '.' || current_schema() AS status;

## Function 1: INSURANCE ABAC FUNCTIONS


In [0]:
CREATE OR REPLACE FUNCTION mask_ssn_last4(ssn STRING) RETURNS STRING
COMMENT 'SSN masking' RETURN CASE WHEN ssn IS NULL THEN ssn ELSE CONCAT('XXX-XX-', RIGHT(REPLACE(ssn, '-', ''), 4)) END;

In [0]:
CREATE OR REPLACE FUNCTION mask_policy_number_last4(policy STRING) RETURNS STRING
COMMENT 'Policy last 4' RETURN CASE WHEN policy IS NULL THEN policy ELSE CONCAT('****', RIGHT(policy, 4)) END;

In [0]:
CREATE OR REPLACE FUNCTION mask_claim_amount_bucket(amt DECIMAL(12,2)) RETURNS STRING
COMMENT 'Claim ranges' RETURN CASE WHEN amt IS NULL THEN 'Unknown' WHEN amt < 1000 THEN '\$0-\$1K'
WHEN amt < 5000 THEN '\$1K-\$5K' WHEN amt < 10000 THEN '\$5K-\$10K' ELSE '\$10K+' END;

In [0]:
CREATE OR REPLACE FUNCTION mask_policyholder_id_hash(id STRING) RETURNS STRING
COMMENT 'Deterministic' RETURN CONCAT('POL_', SUBSTRING(SHA2(id, 256), 1, 12));

In [0]:
SELECT '✅ Insurance functions created!' AS status;

## ✅ Success!

All Insurance masking functions have been created successfully!

### What You Just Created:
- ✅ Masking functions registered in Unity Catalog
- ✅ Functions available for use in SQL queries
- ✅ Foundation for ABAC policies ready

### Verify Your Functions:
You can verify the functions were created by running:
```sql
SHOW FUNCTIONS IN insurance;
```

### 🎯 Next Step:
Continue to **`2_Create_Schema.ipynb`** to create the database tables and load sample data.

---
**Note**: These functions are stored in Unity Catalog and can be used across multiple notebooks and queries.
